# Import the necessary libraries

In [9]:
# import the neccessary libraries
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

# Load the data and perform all the necessary pre-processing

In [2]:
# load the data and label in separate lists
numerals_directory = '/content/drive/MyDrive/Dataset/Numerals'
data = []
label = []

for numeral in os.listdir(numerals_directory):
    numerals_path = os.path.join(numerals_directory, numeral)
    for image in os.listdir(numerals_path):
        image_path = os.path.join(numerals_path, image)
        img = load_img(image_path, target_size=(128, 128))
        img = img_to_array(img)
        data.append(img)
        label.append(int(numeral))

# print("-----Completed-----")

In [3]:
# convert the lists to numpy arrays
data = np.array(data)
label = np.array(label)

In [4]:
# check the size of the data and labels
print(data.shape)
print(label.shape)

(9532, 128, 128, 3)
(9532,)


In [5]:
# converting the numeral labels to array like label
label = to_categorical(label, num_classes=10)

In [6]:
# shuffle the data
n = np.arange(9532)
np.random.shuffle(n)
data_shuffled = data[n]
label_shuffled = label[n]

print("---Shuffling Completed Succesfully---")

---Shuffling Completed Succesfully---


In [7]:
data_shuffled = data_shuffled/255.0

In [8]:
# split the data into train and test data
# train-size: 80%
# val-size: 20%
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.1, stratify=label, random_state=12)

# CUSTOM MODEL

In [27]:
# make a sequential model

model = Sequential()

model.add(Conv2D(64, (7,7), activation='relu', input_shape=(128,128,3)))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(128, (5,5), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [28]:
# compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [29]:
# set callbacks
callback = EarlyStopping(
    monitor='loss',
    patience=3
)

In [30]:
# fit the train data into the model
b = model.fit(
    x_train, y_train,
    epochs=40,
    callbacks=[callback],
    batch_size=32
)

Epoch 1/40
269/269 [==============================] - 28s 92ms/step - loss: 0.1217 - accuracy: 0.8343
Epoch 2/40
269/269 [==============================] - 25s 92ms/step - loss: 0.0038 - accuracy: 0.9986
Epoch 3/40
269/269 [==============================] - 25s 91ms/step - loss: 0.0038 - accuracy: 0.9990
Epoch 4/40
269/269 [==============================] - 25s 92ms/step - loss: 0.0066 - accuracy: 0.9952
Epoch 5/40
269/269 [==============================] - 25s 91ms/step - loss: 0.0028 - accuracy: 0.9986
Epoch 6/40
269/269 [==============================] - 25s 95ms/step - loss: 4.7578e-04 - accuracy: 1.0000
Epoch 7/40
269/269 [==============================] - 25s 92ms/step - loss: 9.3891e-04 - accuracy: 0.9999
Epoch 8/40
269/269 [==============================] - 25s 91ms/step - loss: 0.0089 - accuracy: 0.9952
Epoch 9/40
269/269 [==============================] - 25s 91ms/step - loss: 0.0016 - accuracy: 0.9991


In [35]:
len(b.history['loss'])

9

In [32]:
y_pred_custom = model.predict(x_test)

In [33]:
acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_custom, axis=1))
acc*100

100.0

# PRE-TRAINED MODEL
## 1. MobileNetV2

In [10]:
# create the model
# base model
baseModel = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights="imagenet")

# head model
headModel = baseModel.output
headModel = GlobalAveragePooling2D(name="head_pooling")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(10, activation="softmax")(headModel)

# stack the head model on top of the base model
model1 = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them
for layer in baseModel.layers:
    layer.trainable = False

In [11]:
# compile the model
model1.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [12]:
# set callbacks
callback = EarlyStopping(
    monitor='loss',
    patience=3
)

In [13]:
c = model1.fit(
    x_train, y_train, epochs=40, callbacks=[callback]
)

Epoch 1/40
269/269 [==============================] - 21s 35ms/step - loss: 0.2965 - accuracy: 0.3123
Epoch 2/40
269/269 [==============================] - 9s 34ms/step - loss: 0.2230 - accuracy: 0.5256
Epoch 3/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1908 - accuracy: 0.6153
Epoch 4/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1722 - accuracy: 0.6629
Epoch 5/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1582 - accuracy: 0.6950
Epoch 6/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1494 - accuracy: 0.7168
Epoch 7/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1443 - accuracy: 0.7276
Epoch 8/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1404 - accuracy: 0.7349
Epoch 9/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1300 - accuracy: 0.7578
Epoch 10/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1250 - accur

In [14]:
len(c.history['loss'])

28

In [15]:
y_pred_mobnet = model1.predict(x_test)

In [16]:
acc_mobnet = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_mobnet, axis=1))
acc_mobnet*100

84.80083857442348